# Problem Session 6

The problems in this notebook will cover the content covered in our Inference II lectures including:
- Bootstapping
- Model Specification Testing

In [1]:
import numpy as np
import pandas as pd

#### 1.  Bootstrapping vs ANOVA

We introduce a new dataset which records the following variables for 200 students:

* `read`: Score on a reading test
* `math`: Score on a math test
* `prog`: Categorical variable indicating program of study
    * Takes values `vocational`, `general`, `academic`
* `gre`:  The score on the GRE

In [ ]:
df = pd.read_csv("https://stats.idre.ucla.edu/stat/data/tobit.csv", names = ['id','read', 'math', 'prog', 'gre'], header = 0, index_col= 'id')
df.head()

##### (a) Exploratory Data Analysis

Do a little EDA.  Some ideas include:

* Comparing mean GRE score across different program types.
* Plotting GRE against both "read" and "math" scores, perhaps colored using program type.

Is there any other EDA you can think of?

Did you notice anything interesting in your EDA?

In [ ]:
# Compare mean GRE score by program type


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Make some plots 


##### (b) ANOVA

The mean GRE score for each program type look different, but is this a statistically significant difference?

As mentioned in the Linear Regression Inference lecture, one-way ANOVA is the same as an F-test comparing the two nested models

$$
\begin{align*}
\textrm{Reduced Model: }\textrm{GRE} &= \beta_0 + \epsilon\\
\textrm{Full Model: }\textrm{GRE} &= \beta_0 + \beta_1 \textrm{Vocational} + \beta_2 \textrm{General} + \epsilon
\end{align*}
$$

The $p$-value of an $F$-test comparing the full model to the constant model is included in a `statsmodels model` summary.  So fit the full model to the data and see what you get!

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Hint:  C(prog) gives indicator variables for the levels of prog 
# with one omitted if a constant term is included in the model
model = 

model.summary()

In [ ]:
# Note:  If you would like a traditional "ANOVA table" you can get one like this.abs
# For statsmodels, ANOVA is just a special display 
# of a linear regression model with categorical variables

anova_table = sm.stats.anova_lm(model, typ=2)

# The F statistic and p-value are the same as we obtained in the model summary above.
print(anova_table)

##### (c) Bootstrap confidence intervals

You should have obtained confidence intervals for the mean GRE score of each program type. 

We will now get confidence intervals by bootstrapping instead!

In [ ]:
def bootstrap_ci(data, group_col, outcome_col, n_bootstrap=10000, ci=0.95):
    '''
    Inputs:
        data:  A pandas dataframe.
        group_col: The name of the column with the categorical variable.
        outcome_col:  The name of the column with the continuous outcome.
        n_bootstrap:  The number of bootstrap resamples.  We are resampling the rows of data.
        ci: The nominal coverage of confidence interval.
    
    Outputs:
        group_means:  A dictionary
            keys:  One key for each unique level of group_col
            values:  A tuple (original_mean, ci_lower, ci_upper)
                original_mean: The group outcome mean in the original sample.
                ci_lower:  Lower limit of the confidence interval.
                ci_upper:  Upper limit of the confidence interval.
    '''
    group_means = {}
    lower_percentile = (1 - ci) / 2
    upper_percentile = 1 - lower_percentile
    
    # Get unique groups
    groups = data[group_col].unique()
    
    # Perform percentile bootstrap for each group
    for group in groups:
        group_data = 
        original_mean =    # The original sample mean
        
        # Generate bootstrap samples and means
        # Either loop over range(n_bootstraps) or write vectorized code
        # Note that np.random.choice has a shape parameter which enables vectorization here!
        # In either case you should now have a numpy array of bootstrap means.
        # Call that array bootstrap_means



        # Calculate percentile confidence intervals
        ci_lower = 
        ci_upper = 
        
        group_means[group] = (original_mean, ci_lower, ci_upper)
    
    return group_means

bootstrap_results = bootstrap_ci(df, group_col='prog', outcome_col='gre')

# Print the bootstrap mean and confidence intervals for each group
for group, (mean, ci_lower, ci_upper) in bootstrap_results.items():
    print(f"Group: {group}, Mean: {mean:.2f}, 95% CI (Reverse Bootstrap): [{ci_lower:.2f}, {ci_upper:.2f}]")


#### (2) Regression

In this problem we would like to regress `gre` on the other variables.  We *could* estimate this using vanilla OLS linear regression.  However the GRE has a maximum score of 800.  We can see that 17 of the students did actually achieve this score.  [This meme](https://www.youtube.com/shorts/Lb3lj4IhD0U) has a great point:  these students may have had an aptitude which was greater than what the test could measure.

One tool for estimating the regression coefficients in this circumstance is [Tobit Regression](https://en.wikipedia.org/wiki/Tobit_model).  The idea is that we should model the GRE score as being given by a latent response which is linear in the predictors.  This latent response is then censored at the upper limit of 800.  We then estimate the model parameters using maximum likelihood.

Unfortunately it doesn't seem that statsmodels (or any other Python library) contains an implementation.  So I implemented it from scratch by converting [Michael Clark's implementation in R](https://m-clark.github.io/models-by-example/tobit.html) to Python.

Note:  This shows why understanding theory can be important!  Sometimes you really do need to "roll your own" model.

You can check out the code for this TobitModel class which is found in  `tobit_model.py` in this folder.

Since this is a custom model the API is a bit idiosyncratic, and it is definitely not optimized for either speed or usability.  However it does reproduce the results which Michael Clark was getting using R!

In [8]:
from tobit_model import TobitModel
from sklearn.linear_model import LinearRegression

Let's see how this works on some synthetic data:

In [ ]:
# Creating some synthetic censored data
X = np.linspace(0,10,100)
y = 3 + X + np.random.randn(100)
y = y * (y < 7) + 7 * (y > 7)

# Fitting OLS linear regression model
lr = LinearRegression()
lr.fit(X.reshape(-1,1),y)

# Fitting the Tobit model.  
# Notice what parameters are needed for initialization.
# ul is the (known) censoring upper limit.

# TobitModel requires a design matrix with an initial column of ones.
X_tb = np.ones((100,2))
X_tb[:,1] = X

tb = TobitModel(X = X_tb, y = y, ul = 7)
tb.fit()

plt.scatter(X, y)
plt.plot(X, lr.predict(X.reshape(-1,1)), label = 'OLS fit')
plt.plot(X, tb.predict(X_tb), label = 'Tobit fit')
plt.legend()

plt.show()

In [ ]:
# [constant term, slope, log of variance]
tb.params_

As we can see, the OLS model is inappropriate for censored data while the Tobit model does fine!

##### (a)

Fit the Tobit model on the full dataset:

In [11]:
# Make dummy variables for general and vocational levels of prog
df['general'] = 
df['vocational'] =

# Make a column of ones, needed for my implementation
df['constant'] =

In [12]:
features = ['constant','read', 'math', 'general', 'vocational']

In [13]:
# Instantiate the Tobit Model
model = 

In [14]:
model.fit()

In [ ]:
{features[i]: model.params_[i] for i in range(len(features))}

##### (b) Bootstrapping confidence intervals for the conditional target means.

Since this model was not fit using ordinary least squares, the standard formula for the confidence intervals of conditional target means does not apply.  This is the kind of situation where bootstrapping really shines!

Complete the definition of the following function:

In [16]:
import numpy as np

def bs_conditional_mean(X_train, y_train, ul, X_cond, num_bootstrap_samples):
    '''
    Finds the 'reverse' bootstrap confidence interval for the conditional outcome means of a Tobit model.

    Inputs
    X_train:  A numpy array with initial column of ones.  Shape is (nobs, 1 + number of features).
    y_train:  A numpy array of shape (nobs,).
    ul: Upper limit for Tobit Model.
    X_cond:   Design matrix for the observations we want to condition on.
    num_bootstrap_samples:  Self-explanatory

    Outputs:
    (lower_bound, upper_bound)
    '''
    y_cond = np.zeros((X_cond.shape[0], num_bootstrap_samples))
    
    for i in range(num_bootstrap_samples):
        # sample the indices for bootstrap sampling
        sample_indices = np.random.choice(range(X_train.shape[0]), size=X_train.shape[0], replace=True)
        
        # Slice X_boot using sampled indices
        X_boot = 
        y_boot = 

        # Initialize and fit the Tobit model
        model = 
        model.fit()

        # Store predictions for the conditional mean
        y_cond[:, i] = 
    
    # Calculate the lower and upper bounds of the confidence intervals
    lower_bound =
    upper_bound = 

    return lower_bound, upper_bound


Use this to find the confidence interval for the conditional mean GRE score for

* `read = 60`, `math = 70`, `prog = general`
* `read = 70`, `math = 40`, `prog = vocational`


In [ ]:
df['features']

In [17]:
# Hint:  make a dataframe with the same column names as df[features].
# You can make a dataframe from a dictionary.
X_cond = 

In [18]:
lower_ci, upper_ci = 

In [ ]:
# Confidence interval for mean GRE conditioned on `read = 60`, `math = 70`, `prog = general`
lower_ci[0], upper_ci[0]

In [ ]:
# Confidence interval for mean GRE conditioned on `read = 70`, `math = 40`, `prog = vocational`

lower_ci[1], upper_ci[1]